# GQN With Pool Representation and 8 Generative Steps

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from gqn import GenerativeQueryNetwork, partition
from dataset import GQN_Dataset
from torch.distributions import Normal
import shutil
import os
import numpy as np
import cv2
import torch
import glob

In [ ]:
df = pd.read_csv("main.log")
print("Total steps: {}".format(df["step"].tolist()[-1]))
models = glob.glob("checkpoints/checkpoint_model_*.pth")
models.sort(key=lambda x: os.path.getmtime(x))
print("Last model checkpoint: {}".format(models[-1]))

In [ ]:
df = df[df["step"]>200000]
# df = df[df["step"]>0]
step = df["step"].to_list()
ll = df["ll"].to_list()
kl = df["kl"].to_list()
elbo = df['elbo'].to_list()
sigma = df['sigma'].to_list()
mu = df['mu'].to_list()
kl_av = []
ll_av = []
elbo_av = []

average_step = 1000
for i in range(average_step, len(step)):
    kl_av.append(sum(kl[i-average_step:i])/average_step)
    ll_av.append(sum(ll[i-average_step:i])/average_step)
    elbo_av.append(sum(elbo[i-average_step:i])/average_step)
step = step[average_step:]
sigma = sigma[average_step:]
mu =mu[average_step:]

plt.figure(num=None, figsize=(18, 12))
plt.subplot(231)
plt.plot(step, elbo_av)

plt.subplot(232)
plt.plot(step, ll_av)

plt.subplot(233)
plt.plot(step, kl_av)
plt.subplot(234)
plt.plot(step, mu)
plt.subplot(235)
plt.plot(step, sigma)
plt.show()

In [ ]:
valid_dataset = GQN_Dataset(root_dir="data/rooms-ring-camera/", train=False)
x, v = valid_dataset[1]
x = x.view((1, *x.shape))
v = v.view((1, *v.shape))

max_m=5
x, v, x_q, v_q = partition(x, v, max_m, 5)
batch, *_ = x.shape
print(x.shape)
print(v.shape)
print(x_q.shape)
device = torch.device("cpu")
model = GenerativeQueryNetwork(x_dim=3, v_dim=7, r_dim=256, h_dim=128, z_dim=64, L=8, pool=True).to(device)
model.eval()
checkpoint = torch.load(models[-1])
#checkpoint = torch.load("./checkpoints/checkpoint_model_240000.pth") #
# checkpoint = torch.load("./chkpnts/checkpoint_model_500000.pth") #
model.load_state_dict(checkpoint)

x_mu = model.sample(x, v, v_q)
# x_mu, r, kl = model(x, v, x_q, v_q)
x_mu = x_mu.detach().cpu().numpy()
x_context = x.numpy()
x_query = x_q.numpy()

x_context = np.moveaxis(x_context, [2, 3, 4], [-1, 2, 3])
x_query = np.moveaxis(x_query, [1, 2, 3], [-1, -3, -2])
x_mu = np.moveaxis(x_mu, [1, 2, 3], [-1, -3, -2])

n = 36
for i in range(n):
    full = np.concatenate(x_context[i], axis=1)
    space = np.ones(shape=(full.shape[0], 10, 3))
    full = np.concatenate([full, space, x_mu[i], space, x_query[i]], axis=1)
    plt.figure(num=None, figsize=(18, 6))
    print(i)
    plt.imshow(full)
    plt.show()



In [ ]:
i = 9
#i = 34

In [ ]:
delta = 0.1
new_images = 7
change_n = 256
#change_n = 3
batch = 1

_, _, x_dims, h, w = x.shape

r = model.get_r(x, v)[i]
r = r.cpu().detach().numpy()
r_dims = r.shape
print(r.shape)

view_q = v_q[i].cpu().detach().numpy()
view_q = np.reshape(view_q, (1,  view_q.shape[0]))
view_q = torch.tensor(view_q)

deltas = [delta * (j - new_images//2) for j in range(new_images)]
for neuron_i in range(change_n):
    delta_set = []
    images = []
    for cur_delta in deltas:
        new_r = np.copy(r)
        new_r.astype(np.float16)
        new_r[neuron_i, 0, 0] += cur_delta
        delta_set.append("{:.5}".format(new_r[neuron_i, 0, 0]))
        new_r = np.expand_dims(new_r, axis=0)
        new_r = torch.tensor(new_r)

        device = torch.device("cpu")
        new_r, view_q = new_r.to(device), view_q.to(device)
        torch.manual_seed(8)
        x_mu = model.sample_from_r((h, w), new_r, view_q)
        x_mu = x_mu.detach().cpu().numpy()
        images.append(x_mu)

    images = np.array(images)
    images = np.reshape(images, (new_images, x_dims, h, w))
    images = np.moveaxis(images, [1, 2, 3], [-1, -3, -2])
    images = np.concatenate(images, axis=1)
    print("i: {}".format(neuron_i+1))
    print("      ","        ".join(delta_set))
    plt.figure(num=None, figsize=(21,6))
    plt.imshow(images)
    plt.show()
    


In [ ]:
i=9
delta = 0.1
new_images = 7
change_n = 256
#change_n = 3
batch = 1

_, _, x_dims, h, w = x.shape

r = model.get_r(x, v)[i]
r = r.cpu().detach().numpy()
r_dims = r.shape
print(r.shape)

view_q = v_q[i].cpu().detach().numpy()
view_q = np.reshape(view_q, (1,  view_q.shape[0]))
view_q = torch.tensor(view_q)

deltas = [delta * (j - new_images//2) for j in range(new_images)]
for neuron_i in range(change_n):
    delta_set = []
    images = []
    for cur_delta in deltas:
        new_r = np.copy(r)
        new_r.astype(np.float16)
        new_r[neuron_i, 0, 0] += cur_delta
        delta_set.append("{:.5}".format(new_r[neuron_i, 0, 0]))
        new_r = np.expand_dims(new_r, axis=0)
        new_r = torch.tensor(new_r)

        device = torch.device("cpu")
        new_r, view_q = new_r.to(device), view_q.to(device)
        torch.manual_seed(1)
        x_mu = model.sample_from_r((h, w), new_r, view_q)
        x_mu = x_mu.detach().cpu().numpy()
        images.append(x_mu)

    images = np.array(images)
    images = np.reshape(images, (new_images, x_dims, h, w))
    images = np.moveaxis(images, [1, 2, 3], [-1, -3, -2])
    images = np.concatenate(images, axis=1)
    print("i: {}".format(neuron_i+1))
    print("      ","        ".join(delta_set))
    plt.figure(num=None, figsize=(21,6))
    plt.imshow(images)
    plt.show()
